In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import math
import numpy as np
from guidance import guidance_l1
from drawnow import *
def DCM(psi_rad):
    dcm = np.matrix([[+math.cos(psi_rad), +math.sin(psi_rad)],
                     [-math.sin(psi_rad), +math.cos(psi_rad)]])
    return dcm

In [2]:
# Initial Condition
Hz = 10
dT = 1/Hz
Time = np.arange(0,300,1/Hz)
dT_show = 3.0
Time2show = 0
g = 9.80665
# Target
Xt = np.array([[400,0]])
# Guidance
l1 = guidance_l1()
l1.set_desired_radius(200.0)
l1.set_k_eta(1.0)
l1.set_dir_ccw()
R_ref = l1.get_desired_radius()
# States
Spd_cmd = 10 # Airspeed
Spd = Spd_cmd
psi = 180.0 * math.pi / 180.0
dcm = DCM(psi)
# For plot
a_ = np.array([[0,0]])
V_ = np.array(dcm.dot(np.array([Spd,0])))
X_ = np.array([[0,0]])
psi_ = np.array([psi])
t_ = np.array([0])
Spd_ = np.array([Spd])
G_ = np.array([0])
L_ = np.array([np.linalg.norm(Xt)])
eta_ = np.array([0])

plt.close()
def show_traj():
    plt.subplot(221)
    plt.plot(X_[:,0],X_[:,1],'b')
    plt.plot(Xt[0,0],Xt[0,1],'ro')
    plt.grid()
    plt.xlabel('pN [m]');    plt.ylabel('pE [m]')
    plt.subplot(222)
    plt.plot(t_,G_)
    plt.grid()
    plt.xlabel('Time [sec]');    plt.ylabel('G')
    # plt.plot(t_,Spd_)
    # plt.grid()
    # plt.xlabel('Time [sec]');    plt.ylabel('Speed [m/s')
    plt.subplot(223)
    plt.plot(t_,L_,'b')
    plt.plot([0,t_[-1]],[R_ref, R_ref],'r:')
    plt.grid()
    plt.xlabel('Time [sec]');    plt.ylabel('L [m]')
    plt.subplot(224)
    plt.plot(t_,eta_*57.296)
    # plt.plot(t_[-1],cwccw*10,'ro')
    plt.grid()
    plt.xlabel('Time [sec]');    plt.ylabel('eta [deg]')
    plt.tight_layout()


# for t in Time:
for t in Time:
    # Now..
    target_pos = np.array([Xt[0,0], Xt[0,1], 0])
    my_pos = np.array([X_[-1,0], X_[-1,1],0])
    dX = target_pos - my_pos
    L = np.array([dX[0],dX[1],0])
    V = np.array([V_[-1,0],V_[-1,1],0])
    e_L     = L / np.max([np.linalg.norm(L),0.000001])
    e_V     = V / np.max([np.linalg.norm(V),0.000001])
    Spd = np.sqrt(np.sum(np.square(V)))
    psi = math.atan2(V_[-1,1],V_[-1,0])

    # Guidance Law - Nonlinear Path Following 
    ## Loitering Guidance
    ancmdxy = l1.step_loitering_2d(target_pos, my_pos, V)
    eta     = l1.get_eta()
    aTxy    = 0.1 * np.array([e_V[0], e_V[1]]) * (Spd_cmd - Spd) # Speed Control

    ## Numerical Processing : euler method
    a_   = np.append(a_, [ancmdxy] + aTxy,axis=0)
    V_   = np.append(V_, [V_[-1] + np.array(a_[-1])*dT],axis=0)
    X_   = np.append(X_, [X_[-1] + np.array(V_[-1])*dT],axis=0)
    # States
    G_   = np.append(G_,   np.array([np.linalg.norm(a_[-1])/g]))
    t_   = np.append(t_,   np.array([t]),axis=0)
    L_   = np.append(L_,   np.array([np.linalg.norm(L)]), axis=0)
    Spd_ = np.append(Spd_, np.array([Spd]), axis=0)
    psi_ = np.append(psi_, np.array([psi]),axis=0)
    eta_ = np.append(eta_, np.array([eta]),axis=0)

    # Drawing
    if Time2show+dT_show < t:
        drawnow(show_traj)
        Time2show = Time2show + dT_show

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\bw\.conda\envs\dynamics\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\Users\bw\.conda\envs\dynamics\lib\tkinter\__init__.py", line 749, in callit
    func(*args)
  File "C:\Users\bw\.conda\envs\dynamics\lib\site-packages\matplotlib\backends\_backend_tk.py", line 251, in idle_draw
    self.draw()
  File "C:\Users\bw\.conda\envs\dynamics\lib\site-packages\matplotlib\backends\backend_tkagg.py", line 9, in draw
    super().draw()
  File "C:\Users\bw\.conda\envs\dynamics\lib\site-packages\matplotlib\backends\backend_agg.py", line 436, in draw
    self.figure.draw(self.renderer)
  File "C:\Users\bw\.conda\envs\dynamics\lib\site-packages\matplotlib\artist.py", line 73, in draw_wrapper
    result = draw(artist, renderer, *args, **kwargs)
  File "C:\Users\bw\.conda\envs\dynamics\lib\site-packages\matplotlib\artist.py", line 50, in draw_wrapper
    return draw(artist, rende